In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import webbrowser
import tkinter as tk
from tkinter import ttk
from scipy.spatial import distance

def detect_face(image, face_mesh):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)
    return results.multi_face_landmarks[0] if results.multi_face_landmarks else None

def analyze_skin(image, face_landmarks):
    height, width, _ = image.shape
    skin_tone_points = [10, 234, 454, 152]
    colors = [image[int(face_landmarks.landmark[idx].y * height), int(face_landmarks.landmark[idx].x * width)] for idx in skin_tone_points]
    avg_color = np.mean(colors, axis=0)
    return classify_skin_tone(avg_color)

def classify_skin_tone(color):
    color_wheel = {"Fair": [240, 200, 180], "Medium": [200, 150, 120], "Deep": [130, 80, 60]}
    return min(color_wheel, key=lambda tone: distance.euclidean(color, color_wheel[tone]))

def detect_eye_shape(face_landmarks):
    left_eye_top = face_landmarks.landmark[159]
    left_eye_bottom = face_landmarks.landmark[145]
    left_eye_width = face_landmarks.landmark[33]
    left_eye_outer = face_landmarks.landmark[133]
    eye_height = abs(left_eye_top.y - left_eye_bottom.y)
    eye_width = abs(left_eye_outer.x - left_eye_width.x)
    return "Almond Eyes" if eye_width / eye_height > 3 else "Hooded Eyes" if eye_height / eye_width > 0.4 else "Round Eyes"

def detect_dark_circles(image, face_landmarks):
    return "Use peach/orange corrector if dark circles detected"

def recommend_makeup(skin_tone, eye_shape):
    return {"foundation": f"Best foundation for {skin_tone} skin", "eye makeup": f"Best styles for {eye_shape}", "lip color": "Choose nude, pink, or berry shades", "contouring": "Apply contour on jawline, cheekbones, and nose bridge"}

def proceed_to_selection():
    window = tk.Tk()
    window.title("Brand Selection")
    window.geometry("300x200")
    tk.Label(window, text="Select a brand:").pack()
    brands = ["Maybelline", "MAC", "Loreal", "Fenty Beauty"]
    selected_brand = tk.StringVar()
    brand_menu = ttk.Combobox(window, textvariable=selected_brand, values=brands)
    brand_menu.pack()
    def open_brand_page():
        brand_links = {"Maybelline": "https://www.maybelline.com/", "MAC": "https://www.maccosmetics.com/", "Loreal": "https://www.lorealparisusa.com/", "Fenty Beauty": "https://fentybeauty.com/"}
        webbrowser.open(brand_links.get(selected_brand.get(), "https://www.google.com"))
    tk.Button(window, text="Proceed", command=open_brand_page).pack()
    window.mainloop()

mp_face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)
capture = cv2.VideoCapture(0)
while True:
    ret, frame = capture.read()
    if not ret:
        break
    face_landmarks = detect_face(frame, mp_face_mesh)
    if face_landmarks:
        skin_tone = analyze_skin(frame, face_landmarks)
        eye_shape = detect_eye_shape(face_landmarks)
        dark_circle_advice = detect_dark_circles(frame, face_landmarks)
        makeup_recommendations = recommend_makeup(skin_tone, eye_shape)
        y_offset = 30
        for text in [dark_circle_advice] + [f"{key}: {value}" for key, value in makeup_recommendations.items()]:
            cv2.putText(frame, text, (30, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            y_offset += 20
    cv2.imshow("Face Analysis", frame)
    if cv2.waitKey(1) & 0xFF == ord('p'):
        capture.release()
        cv2.destroyAllWindows()
        proceed_to_selection()
        break
capture.release()
cv2.destroyAllWindows()

I0000 00:00:1738591448.874704 24940266 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1738591448.886391 24941440 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1738591448.891041 24941445 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1738591450.217907 24941441 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
